In [1]:
import boto3
import json
import time
import pandas as pd
from utils import *

# This notebook will submit a batch job that will optimize the strategy created

In [2]:
TOTAL_JOBS = "6" # How many jobs in parallel we want to run. Could be explained as: In how many equally sized subspaces we want to divide the original space?

JOB_DEFINITION = 'arn:aws:batch:us-west-1:902310698465:job-definition/backtester_job_definition:4' # ARN for the job difinition (always the same)

JOB_QUEUE = 'arn:aws:batch:us-west-1:902310698465:job-queue/backtester_queue' # ARN for the job queue (always the same)

JOB_NAME = 'NEW_ERA_MAUI' # Could be anything, final and partial files will contain this name

N_CPUS = "3" # How many cpus we want to instantiate to run all the jobs

In [3]:
# We need to create the dictionary containing each indicator name and the range or list of values for each, here is an example:
# MUST HAVE SAME NAMES AS IN THE STRATEGY FILE
DICTIONARY = {'EMA5':[15,30,45],'low5':[5,30], 'diff%':[10], 'rsi5':[14], 'BB5':[10,20],'BB1':[20], 'rsi1':[14]}

# We need it as string
DICTIONARY_STRING = f"{DICTIONARY}"

In [4]:
df = submit_job(DICTIONARY_STRING,
               TOTAL_JOBS,
               JOB_DEFINITION,
               JOB_QUEUE, 
               JOB_NAME,
               N_CPUS,
               TIMEOUT=20000
               )

Job submitted with ID: 1763ba71-35e3-4187-bd98-2a1ac57559f3
Job status is PENDING
Job succeeded


In [5]:
df = collect_results('NEW_ERA_MAUI', info=True)
df.drop_duplicates(inplace=True)

In [136]:
def high_low(x, low, high, threshold=0.3):

    if x < -1 and high < threshold:
        return True
    elif x > 1 and low > -threshold:
        return True
    elif x < -1 and high > threshold:
        return False
    elif x > 1 and low < -threshold:
        return False
    else:
        return 'neutral'

In [139]:
df.sort_values('time_buy')

,time_buy,price_buy,time_sold,price_sold,%,low,high,crypto,combination
73849,2022-06-01 07:00:00,31841.33,2022-06-01 07:55:00,31936.54,0.299014,-0.285572,0.322160,btc,[ 0 0 0 -1 0 -1 0 -1 -1 0]
1559,2022-06-01 07:00:00,31841.33,2022-06-01 07:55:00,31936.54,0.299014,-0.285572,0.322160,btc,[-1 -1 -1 -1 0 -1 0 -1 0 -1]
47604,2022-06-01 07:00:00,31841.33,2022-06-01 07:55:00,31936.54,0.299014,-0.285572,0.322160,btc,[ 0 -1 -1 -1 0 0 -1 0 -1 -1]
19751,2022-06-01 07:00:00,31841.33,2022-06-01 07:55:00,31936.54,0.299014,-0.285572,0.322160,btc,[-1 0 0 -1 -1 -1 0 -1 0 -1]
53293,2022-06-01 07:00:00,31841.33,2022-06-01 07:55:00,31936.54,0.299014,-0.285572,0.322160,btc,[ 0 -1 -1 -1 0 0 0 -1 -1 -1]
...,...,...,...,...,...,...,...,...,...
50338,2023-03-01 06:30:00,23104.28,2023-03-01 07:00:00,23178.71,0.322148,0.000000,0.322148,btc,[ 0 -1 -1 -1 0 0 -1 1 0 -1]
551,2023-03-01 06:30:00,23104.28,2023-03-01 07:00:00,23178.71,0.322148,0.000000,0.322148,btc,[-1 -1 -1 -1 0 1 0 1 0 0]
47603,2023-03-01 07:00:00,23178.71,2023-03-01 07:00:00,23178.71,0.000000,0.000000,0.000000,btc,[ 1 0 0 -1 -1 0 -1 -1 -1 0]
45805,2023-03-01 07:00:00,23178.71,2023-03-01 07:00:00,23178.71,0.000000,0.000000,0.000000,btc,[ 1 0 0 -1 -1 0 -1 -1 -1 -1]


In [152]:
grouped = df.groupby('combination')['result'].value_counts(normalize=True).unstack()
grouped['total'] = grouped[True] + grouped[False] + grouped['neutral']

In [153]:
grouped.sort_values(True, ascending=False)

result,False,True,neutral,total
combination,,,,
[ 0 -1 0 -1 0 1 0 1 0 1],0.036496,0.248175,0.715328,1.0
[ 0 0 -1 0 0 1 0 1 0 1],0.041379,0.241379,0.717241,1.0
[-1 -1 -1 -1 -1 0 0 1 -1 1],0.034722,0.222222,0.743056,1.0
[ 0 1 1 1 0 1 0 1 0 -1],0.012019,0.204327,0.783654,1.0
[ 1 1 1 1 0 1 -1 -1 -1 1],0.008929,0.196429,0.794643,1.0
...,...,...,...,...
[-1 1 1 0 -1 0 1 -1 -1 -1],0.001992,0.075697,0.922311,1.0
[-1 1 0 0 0 0 1 -1 -1 -1],0.001852,0.074074,0.924074,1.0
[-1 1 -1 1 -1 0 -1 -1 0 -1],0.001767,0.068905,0.929329,1.0


In [140]:
df['result'] = df.apply(lambda row: high_low(row['%'], row['low'], row['high']), axis=1)
df_not = df[df.result != 'neutral']
df_not.shape[0] / df.shape[0] * 100

In [118]:
df_not['result'].sum() / df_not.shape[0]

0.9004225809946906

In [55]:
df_not

,time_buy,price_buy,time_sold,price_sold,%,low,high,crypto,result
0,2022-06-01 21:15:00,31430.01,2022-06-01 21:45:00,31101.78,-1.044320,-1.044320,0.114031,btc,True
1,2022-06-03 17:00:00,30221.45,2022-06-03 17:35:00,29900.78,-1.061068,-1.061068,0.000000,btc,True
2,2022-06-06 07:00:00,29933.82,2022-06-06 07:45:00,30298.14,1.217085,0.000000,1.217085,btc,True
3,2022-06-07 07:00:00,31259.71,2022-06-07 07:20:00,30729.00,-1.697744,-1.697744,0.142516,btc,True
4,2022-06-10 07:05:00,30087.29,2022-06-10 07:15:00,29700.00,-1.287221,-1.287221,0.000000,btc,True
...,...,...,...,...,...,...,...,...,...
71929,2023-02-17 23:25:00,24026.57,2023-02-18 00:05:00,24272.39,1.023117,-0.328220,1.023117,btc,False
71930,2023-02-20 16:00:00,24473.22,2023-02-20 16:40:00,24768.56,1.206788,-0.143545,1.206788,btc,True
71933,2023-02-23 09:00:00,24210.98,2023-02-23 09:40:00,24537.62,1.349140,-0.104828,1.349140,btc,True
71937,2023-02-27 01:05:00,23259.77,2023-02-27 01:50:00,23505.97,1.058480,-0.156794,1.058480,btc,True


In [8]:
dff = collect_results('NEW_ERA_MAUI')

In [9]:
best = dff.sort_values('differ',ascending=False).head(40)
best.reset_index(drop=True, inplace=True)
best.to_csv('best_combi.csv')
best

,0,1,2,3,4,5,6,7,8,9,total,vol,differ
0,0,0,0,0,-1,0,1,1,1,-1,181,56,30.94
1,0,0,0,0,-1,0,1,1,1,0,181,56,30.94
2,0,0,0,0,-1,-1,1,1,1,-1,181,56,30.94
3,0,0,0,0,-1,-1,1,1,1,0,181,56,30.94
4,0,0,-1,0,-1,0,1,1,1,0,155,47,30.32
5,0,0,-1,0,-1,0,1,1,1,-1,155,47,30.32
6,0,0,-1,0,-1,-1,1,1,1,0,155,47,30.32
7,0,0,-1,0,-1,-1,1,1,1,-1,155,47,30.32
8,0,0,0,0,-1,-1,0,1,1,-1,261,79,30.27
9,0,0,0,0,-1,-1,0,1,1,0,261,79,30.27


In [20]:
best = dff.sort_values('differ',ascending=False).head(40)
best.reset_index(drop=True, inplace=True)
best

,0,1,2,3,4,5,6,7,8,9,10,11,total,vol,differ
0,1,0,0,0,0,-1,-1,0,-1,0,-1,1,117,55,47.01
1,1,0,0,0,0,-1,0,0,-1,0,0,1,117,55,47.01
2,1,0,0,0,0,-1,-1,0,-1,0,0,1,117,55,47.01
3,1,0,0,0,0,-1,0,0,-1,0,-1,1,117,55,47.01
4,1,0,0,0,0,-1,-1,0,-1,-1,0,1,115,54,46.96
5,1,0,0,0,0,-1,0,0,-1,-1,0,1,115,54,46.96
6,1,0,0,0,0,-1,0,0,-1,-1,-1,1,115,54,46.96
7,1,0,0,0,0,-1,-1,0,-1,-1,-1,1,115,54,46.96
8,-1,0,0,0,1,0,0,0,-1,0,-1,1,112,52,46.43
9,-1,0,0,0,1,0,0,0,-1,0,0,1,112,52,46.43


In [31]:
dff[(dff['7'] == 1)&(dff['8'] == 1)&(dff['9'] == 1)]

,0,1,2,3,4,5,6,7,8,9,total,vol,differ
140,-1,-1,-1,-1,1,1,-1,1,1,1,122,17,13.93
164,-1,-1,-1,-1,1,1,0,1,1,1,122,17,13.93
189,-1,-1,-1,-1,1,0,-1,1,1,1,157,19,12.10
214,-1,-1,-1,-1,1,0,0,1,1,1,157,19,12.10
290,-1,-1,-1,-1,0,1,-1,1,1,1,122,17,13.93
...,...,...,...,...,...,...,...,...,...,...,...,...,...
20153,0,0,0,0,1,0,0,1,1,1,759,164,21.61
20231,0,0,0,0,0,1,-1,1,1,1,773,172,22.25
20257,0,0,0,0,0,1,0,1,1,1,773,172,22.25
20284,0,0,0,0,0,0,-1,1,1,1,816,177,21.69
